# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from os import listdir
import time
import cv2
import json

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# Setup:

Specify paths in the next cell

In [6]:
# This is needed to display the images.
%matplotlib inline

path = "/Users/ninawiedemann/Desktop/UNI/Praktikum/models/research/object_detection/"
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("/Users/ninawiedemann/Desktop/UNI/Praktikum/models/research/")
sys.path.append(path)

# PATH_TO_TEST_IMAGES_DIR = 'test_images/video3'
# TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, image_x) for image_x in listdir(PATH_TO_TEST_IMAGES_DIR)]
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]
# print(TEST_IMAGE_PATHS)

batter_videos = "/Volumes/Nina Backup/high_quality_testing/batter/"
name = "#20 Dylan Cook (5).mp4"

INPUT_VIDEO_PATH = os.path.join(batter_videos, name)

# IF THE OUTPUT IMAGES SHOULD BE SAVED: Set first variable true and specify output directory
SHOW_IMAGES = True
SAVE_IMAGES = False
TEST_OUT_PATH = "output_examples/video5/"
if not os.path.exists(TEST_OUT_PATH):
    os.makedirs(TEST_OUT_PATH)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [10]:
# What model to download.
MODEL_NAME = "rfcn_resnet101_coco_11_06_2017"
# fourth_best: "rfcn_resnet101_coco_11_06_2017"
# third_best: "faster_rcnn_resnet101_coco_11_06_2017"
# best ging nicht: "faster_rcnn_nas_coco_24_10_2017"
# second_best: "faster_rcnn_inception_resnet_v2_atrous_coco_11_06_2017"
#worst: 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = path+ MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join(path, 'data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model - does not need to be executed for most of the models because already loaded

In [58]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [8]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [11]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

haaalo <class 'object_detection.protos.string_int_label_map_pb2.StringIntLabelMap'> 


## Helper code

In [12]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

# Run script on video

In [13]:
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)

output_dic = {}

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        frame_count = 0
        while frame_count<3:
            ret, image_np = cap.read()
            if image_np is None:
                break
            # image_np = frame 
            frame_count+=1
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            print(image_np_expanded.shape)
            # Actual detection.
            tic = time.time()
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            print(boxes.shape, scores.shape, classes.shape, num.shape)
            # Visualization of the results of a detection.
            print("Time for session: ", time.time()-tic)
            
            index = np.union1d(np.where(np.squeeze(classes)==40)[0],np.where(np.squeeze(classes)==39)[0]) #np.where(np.squeeze(classes)!=1)[0]
            # print(index)
            classes = np.squeeze(classes)[index]
            scores = np.squeeze(scores)[index]
            boxes = np.squeeze(boxes)[index]
            num = len(classes)
            print("boxes", boxes*np.array([[1080,1920,1080, 1920] for _ in range(len(boxes))]))
            print("scores", scores)
            print("classes", classes)
            
            inner_dic = {}
            for i, s in enumerate(scores):
                if s>0.6:
                    if classes[i]==39:
                        class_name = "bat"
                    else:
                        class_name = "glove"
                    inner_dic[name] = {"score":float(s), "box":boxes[i].tolist()}
            output_dic[str(frame_count).zfill(4)] = inner_dic
            if SHOW_IMAGES:
                vis_util.visualize_boxes_and_labels_on_image_array(
                  image_np,
                  boxes, classes.astype(np.int32), scores,
                  category_index,
                  use_normalized_coordinates=True,
                    min_score_thresh = 0.6,
                    max_boxes_to_draw = 100,
                  line_thickness=8)
                plt.figure(figsize=IMAGE_SIZE)
                plt.imshow(image_np)
                plt.show()
                if SAVE_IMAGES:
                    cv2.imwrite(TEST_OUT_PATH+image_path.split("/")[-1], image_np)
                    #plt.savefig()
print(output_dic)
with open("outputs/"+name+ ".json", "w") as outfile:
    json.dump(output_dic, outfile)

(1, 1080, 1920, 3)
(1, 300, 4) (1, 300) (1, 300) (1,)
Time for session:  30.724731922149658
boxes [[ 168.53264451  729.15985107  364.35690522  821.01768494]
 [ 677.08420515  241.73415184  760.98715782  345.43604851]
 [ 155.99941134  698.86922836  414.27470326  820.17419815]
 [ 302.63798833  127.27981567  384.51003671  212.003088  ]
 [ 671.50036097  252.49986649  747.5973773   313.31726074]
 [ 266.57175064  711.13283157  286.12627745  723.89505386]
 [ 260.31583607  123.31749916  388.00043821  230.00121117]
 [ 249.33273733  710.87596893  274.9790597   726.1390686 ]
 [ 272.34169722   71.47946119  389.21683073  259.3415451 ]
 [ 166.54306769   73.20828438  721.85814857  939.95487213]
 [ 587.16501474  115.13846397  713.99549961  189.83267784]
 [ 676.58537865  179.66302872  850.61827898  372.53099442]
 [ 652.23383904  154.97573376  794.12497044  374.72963333]
 [ 198.52781475  700.68511963  225.29533803  726.10771179]
 [ 206.24718547   29.18799877  644.59836245  262.17418671]
 [ 545.07413864  

NameError: name 'IMAGE_SIZE' is not defined

In [ ]:
print(np.where(classes==40)[0])
print(np.where(classes==39)[0])
print(classes)
print(np.union1d(np.where(classes==40)[0],np.where(classes==39)[0]))

In [43]:
print(classes[np.where(np.union1d(classes== 47, classes== 39))[0]])

[ 39.]


In [23]:
index = np.where(np.squeeze(classes)!=1)[0]
print(index)
classes = np.squeeze(classes)[index]
scores = np.squeeze(scores)[index]
boxes = np.squeeze(boxes)[index]
num = len(classes)
print(boxes.shape, scores.shape, classes.shape, num)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48]
(49, 4) (49,) (49,) 49


In [ ]:
name = "#20 Dylan Cook (5).mp4" #"#22 Jordan Winawer (3).mp4"
path = "/Volumes/Nina Backup/high_quality_testing/batter/"
# "/Users/ninawiedemann/Desktop/UNI/Praktikum/high_quality_testing/batter/"
cap = cv2.VideoCapture(path+name)
p=0
while True:
    ret, frame = cap.read()
    if frame is None:
        break
    cv2.imwrite("/Users/ninawiedemann/Desktop/UNI/Praktikum/models/research/object_detection/test_images/video3/pic"+str(p)+".jpg", frame)
    p+=1

In [ ]:
import cv2
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS[120:170]:
            if image_path[-4:] != ".jpg":
                continue
            print(image_path, TEST_OUT_PATH+image_path.split("/")[-1])
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            print(image_np_expanded.shape)
            # Actual detection.
            tic = time.time()
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            print(boxes.shape, scores.shape, classes.shape, num.shape)
            # Visualization of the results of a detection.
            print("Time for session: ", time.time()-tic)
            
            index = np.union1d(np.where(np.squeeze(classes)==40)[0],np.where(np.squeeze(classes)==39)[0]) #np.where(np.squeeze(classes)!=1)[0]
            print(index)
            classes = np.squeeze(classes)[index]
            scores = np.squeeze(scores)[index]
            boxes = np.squeeze(boxes)[index]
            num = len(classes)
            
            print(scores)
            print(classes)

            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              boxes, classes.astype(np.int32), scores,
              category_index,
              use_normalized_coordinates=True,
                min_score_thresh = 0.001,
                max_boxes_to_draw = 100,
              line_thickness=8)
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            plt.show()
            cv2.imwrite(TEST_OUT_PATH+image_path.split("/")[-1], image_np)
            #plt.savefig()

# old faster rcnn code

In [ ]:
CLASSES = ('background',
'person', 'bicycle', 'car','motorcycle','airplane','bus','train', 'truck', 'boat','traffic light',
'fire hydrant', 'stop sign', 'parking meter', 'bench','bird','cat','dog','horse','sheep',
'cow', 'elephant','bear','zebra','giraffe','hat','umbrella', 'handbag','tie','suitcase',
'frisbee','skis','snowboard','sports ball','kite', 'baseball bat','baseball glove','skateboard','surfboard','tennis racket',
'bottle','wine glass','cup','fork','knife','spoon','bowl','banana','apple','sandwich',
'orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','couch','potted plant',
'bed','dining table','window','tv','laptop','mouse','remote','keyboard','cell phone','microwave',
'oven', 'sink','refrigerator','blender','book','clock','vase','scissors','teddy bear','hair drier','tooth brush')

def vis_detections(im, class_name, dets, thresh):
    
    """Draw detected bounding boxes."""
    inds = np.where(dets[:, -1] >= thresh)[0]
    #if len(inds) == 0:
    #    return

    im = im[:, :, (2, 1, 0)]
    fig, ax = plt.subplots(figsize=(6, 6))
    
    ax.imshow(im, aspect='equal')
    
    for i in inds:
        bbox = dets[i, :4]
        score = dets[i, -1]

        ax.add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2] - bbox[0],
                          bbox[3] - bbox[1], fill=False,
                          edgecolor='red', linewidth=3.5)
            )
    
        ax.text(bbox[0], bbox[1] - 2,
                '{:s} {:.3f}'.format(class_name, score),
                bbox=dict(facecolor='blue', alpha=0.5),
                fontsize=14, color='white')

    ax.set_title(('{} detections with '
                  'p({} | box) >= {:.1f}').format(class_name, class_name,
                                                  thresh),
                  fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.draw()
    plt.show()
    
    return bbox[0], bbox[1], bbox[2], bbox[3]
        
centerBat = []
batBox = []
isBat=[0]*9
counter=0
from os import listdir
path = path_all + "outputpic/"
for f in sorted(listdir(path)):
    if ((f[0]!=".") and (int(f[7]) ==6)):
        # print(f[:4])
        im = plt.imread(path_all+"inputpic/"+f[:4]+".jpg")
        if f == "0102_0.6":
            fra = im
        with open(path+f, "r") as infile:
            try:
                dets_dic = json.load(infile)
            except ValueError:
                continue
        CONF_THRESH = float(f[5:])
        NMS_THRESH = 0.2

        for cls_ind, cls in enumerate(CLASSES[35:36]):
            cls_ind += 1 # because we skipped background
            try:
#                 if int(f[7])==6:
                dets = dets_dic[cls]
                x1, y1, x2, y2 = list(vis_detections(im, cls, np.array(dets), thresh=CONF_THRESH))
                batBox.append([np.array([[x1,y1],[x2,y2]]), int(f[:4])-1])
#                 centerBat.append([(x2+x1)/2, (y2+y1)/2, int(f[:4])-1])

                isBat[int(f[7])-1]+=1
                counter+=1
                # print("detection", counter)
            except KeyError:
                # print("no detection", counter)
                continue
                
batBox = np.array(batBox)
batLenCap = max(np.linalg.norm(batBox[i,0][0]-batBox[i,0][1]) for i in range(len(batBox)))
oldBatBox = batBox

In [ ]:
# batTip = []
# batBase = []
# # wristBase = []
# for i in range(len(batBox)):
#     frame = batBox[i][1]
# #     wristBase.append([wristTracker[frame][0], wristTracker[frame][1],frame])
#     if abs(batBox[i][0][0]-wristTracker[frame][0]) > abs(batBox[i][0][2]-wristTracker[frame][0]):
#         xTipInd = 0
#     else:
#         xTipInd = 2
#     if abs(batBox[i][0][1]-wristTracker[frame][1]) > abs(batBox[i][0][3]-wristTracker[frame][1]):
#         yTipInd = 1
#     else:
#         yTipInd = 3
#     batTip.append([batBox[i][0][xTipInd],batBox[i][0][yTipInd], frame])
#     batBase.append([batBox[i][0][abs(xTipInd-2)],batBox[i][0][abs(yTipInd-4)], frame])
    
# tip = []
# base = []
# for i in range(len(batTip)):
#      if batTip[i][2] in range(80,150):
#         tip.append(batTip[i])
#         base.append(batBase[i])

# tip = np.array(tip)
# base = np.array(base)
# batLenCap = max(np.linalg.norm(tip[i][:2]-base[i][:2]) for i in range(len(tip)))

### FMO C

In [ ]:
BASE = "/Volumes/Nina Backup/ColeLoncar/" # os.path.join('BatDetectHQ', 'ColeLoncar')
VIDEO = os.path.join(BASE, 'inputvid/test.mp4')
# IMG = os.path.join(BASE,'inputpic/')
IMGS = "/Volumes/Nina Backup/ColeLoncar/inputpic/"
length = 300
# We read the images as grayscale so they are ready for thresholding functions.
images = []
images = [cv2.imread(IMGS+"%04d.jpg"%idx, cv2.IMREAD_GRAYSCALE) for idx in range(1,length+1)] #IMG_TEMPLATE.format(idx), ) 
#fig = plt.figure(figsize=(18, 16), edgecolor='k')
#plt.imshow(images[162])
#plt.show()

# blurBatTip = []
# blurBatBase = []

blurBat = []

candidate_values = []
whiteness_values = []
location = []
frame_indizes = []
t=0
frame_before_close_wrist = False

# currentTip = tip[0,:2]
# currentBase = base[0,:2]
# currentInd = tip[0,2]
currentInd = batBox[0,1]
inders = []

for t in missingInd:#range(1, length-2):
    for ind in range(int(currentInd), t):
        if ind in batBox[:,1]:
            loca = np.where(batBox[:,1]==ind)
            loca = loca[0][0]
            currentBat = batBox[loca,0]

#         if ind in tip[:,2]:
#             loca = np.where(tip[:,2]==ind)
#             loca = loca[0][0]
#             currentTip = tip[loca,:2]
#             currentBase = base[loca,:2]
#             currentInd = ind

    im_tm1 = images[t - 1]
    im_t = images[t]
    im_tp1 = images[t + 1]
    
#     print images[t-1].shape
#     print images[t].shape
#     print images[t+1].shape
    print(t)
    delta_plus = cv2.absdiff(im_t, im_tm1)
    delta_0 = cv2.absdiff(im_tp1, im_tm1)
    delta_minus = cv2.absdiff(im_t,im_tp1)
    sp = cv2.meanStdDev(delta_plus)
    sm = cv2.meanStdDev(delta_minus)
    s0 = cv2.meanStdDev(delta_0)
    #print("E(d+):", sp, "\nE(d-):", sm, "\nE(d0):", s0)


    th = [
        sp[0][0, 0] + 3 * math.sqrt(sp[1][0, 0]),
        sm[0][0, 0] + 3 * math.sqrt(sm[1][0, 0]),
        s0[0][0, 0] + 3 * math.sqrt(s0[1][0, 0]),
    ]

    
    # OPENCV THRESHOLD
    start = time.time()
    ret, dbp = cv2.threshold(delta_plus, th[0], 255, cv2.THRESH_BINARY)
    ret, dbm = cv2.threshold(delta_minus, th[1], 255, cv2.THRESH_BINARY)
    ret, db0 = cv2.threshold(delta_0, th[2], 255, cv2.THRESH_BINARY)

    detect = cv2.bitwise_not(cv2.bitwise_and(cv2.bitwise_and(dbp, dbm), 
                    cv2.bitwise_not(db0)))

    ocv_time = (time.time() - start) * 1000
    
    
    # CONNECTED BOX
    # The original `detect` image was suitable for display, but it is "inverted" and not suitable
    # for component detection; we need to invert it first.
    start = time.time()
    nd = cv2.bitwise_not(detect)
    num, labels, stats, centroids = cv2.connectedComponentsWithStats(nd, ltype=cv2.CV_16U)


    # We set an arbitrary threshold to screen out smaller "components"
    # which may result simply from noise, or moving leaves, and other
    # elements not of interest.
    min_area = 500


    d = detect.copy()
    candidates = list()
    for stat in stats[1:]:
        area = stat[cv2.CC_STAT_AREA]
        if area < min_area:
            continue # Skip small objects (noise)

        lt = (stat[cv2.CC_STAT_LEFT], stat[cv2.CC_STAT_TOP])
        rb = (lt[0] + stat[cv2.CC_STAT_WIDTH], lt[1] + stat[cv2.CC_STAT_HEIGHT])                                                
        bottomLeftCornerOfText = (lt[0], lt[1] - 15)

        candidates.append((lt, rb, area))

    
    #plt.figure(figsize=(10, 10), edgecolor='k')
    
    # MOTION MODEL
    
    # Thinning threshold.
    psi = 0.7

    # Area matching threshold.
#     gamma = 0.4
    gamma = 20
    area_max = 0
#     area_min = 1

    fom_detected = False
    start = time.time()
    col = len(candidates)
    sub_regions = list()
    
    
    index = []
    close_to_wrist = False

    for i, candidate in enumerate(candidates):
        # The first two elements of each `candidate` tuple are
        # the opposing corners of the bounding box.
        x1, y1 = candidate[0]
        x2, y2 = candidate[1]
        
        center = [(x1+x2)/2,(y1+y2)/2]

        actual_area = candidate[2]
        cand = nd[y1:y2, x1:x2]
        dt = cv2.distanceTransform(cand, distanceType=cv2.DIST_L2, maskSize=cv2.DIST_MASK_PRECISE)
        radius = np.amax(dt)
        ret, Pt = cv2.threshold(dt, psi * radius, 255, cv2.THRESH_BINARY)

        w = x2 - x1
        h = y2 - y1
        path_len = math.sqrt(w * w + h * h)
        expected_area = radius * (2 * path_len + math.pi * radius)

        area_ratio = abs(actual_area / expected_area - 1)

        location.append(center)
        index.append(i)
        area = candidates[i][2]
        candidate_values.append(area_ratio)
        patch = im_t[y1:y2, x1:x2]
        whiteness_values.append(np.mean(patch))
        frame_indizes.append(t-1)   
    
    
    if len(candidates)>0:
        print("DETECTED", t)#, whiteness_values[-1], candidate_values[-1])
        inders.append(t)
        plt.figure(figsize=(10, 10), edgecolor='r')
        candidates = combineOverlapping(candidates)
#         print candidates
        img = np.tile(np.expand_dims(im_t.copy(), axis = 2), (1,1,3))
        for fom in index:
            cv2.rectangle(img, candidates[fom][0], candidates[fom][1],[255,0,0], 4)
#         wristTup = (int(wristTracker[t][0])-10,int(wristTracker[t][1])-10)
#         wristTupA = (int(wristTracker[t][0])+10,int(wristTracker[t][1])+10)
#         cv2.rectangle(img, wristTup , wristTupA, [0,255,0], 4)

        plt.imshow(img, 'gray')
        plt.title("Detected FOM".format(t))
        plt.show()
        
        newBox = closestBox(candidates, currentBat)
        if newBox.any():
            blurBat.append([np.array(newBox), int(t)])
        
#         theTip, theBase = findBatVerts(candidates,currentTip)
            
#         if (theTip, theBase) != (-1,-1):
#             blurBatTip.append([theTip[0],theTip[1], t])
#             blurBatBase.append([theBase[0],theBase[1], t])
#             currentTip = theTip

In [ ]:
# def findBatVerts(cands,tipVert):
#     minVal = 2*batLenCap
#     minInd = (-1, -1)
#     for i in range(len(cands)):
#         rec = np.array(cands[i][:2])
#         for xInd in range(2):
#             xVal = rec[xInd,0]
#             for yInd in range(2):
#                 yVal = rec[yInd,1]
#                 if np.linalg.norm(tipVert - [xVal,yVal]) < minVal:
#                     minVal = np.linalg.norm(tipVert - [xVal,yVal])
#                     minInd = xInd, yInd
#                     candLoc = i
#     if minInd != (-1,-1):
#         rec = np.array(cands[candLoc][:2])
#         newTip = rec[minInd[0], 0], rec[minInd[1], 1]
#         newBase = rec[abs(minInd[0]-1), 0], rec[abs(minInd[1]-1), 1]
#         return newTip, newBase
#     else:
#         return -1, -1
      
    
# blurTip = np.array(blurBatTip)
# blurBase = np.array(blurBatBase)
# combTip = np.concatenate((tip,blurTip),axis=0)
# combTip =  combTip[combTip[:,2].argsort()]
# combBase = np.concatenate((base,blurBase),axis=0)
# combBase =  combBase[combBase[:,2].argsort()]



In [ ]:
### 2D plotting of base and tip
plt.plot(base[:,0], base[:,1])
plt.ylim(650,300)

for i in range(len(base)):
    plt.annotate(base[i][2], (base[:,0][i], base[:,1][i]))

plt.xlabel()
plt.show()

### Display wrist, base and tip images in the end

In [ ]:
i = -1

for t in tip[:,2]:
    i += 1
    print("DETECTED", t)
    im_t =images[int(t)]
#     loca = np.where(tip[:,2]==int(t))
#     if (len(loca[0]) == 0):
#         color = [0,0,255]
#         loca = np.where(blurTip[:,2]==int(t))
#         loca = loca[0][0]
#         tipTup = (int(blurTip[loca][0])-10,int(blurTip[loca][1])-10)
#         tipTupA = (int(blurTip[loca][0])+10,int(blurTip[loca][1])+10)
#         baseTup = (int(blurBase[loca][0])-10,int(blurBase[loca][1])-10)
#         baseTupA = (int(blurBase[loca][0])+10,int(blurBase[loca][1])+10)
        
#     else:
#         color = [255,0,0]
#         loca = loca[0][0]
#         tipTup = (int(tip[loca][0])-10,int(tip[loca][1])-10)
#         tipTupA = (int(tip[loca][0])+10,int(tip[loca][1])+10)
#         baseTup = (int(base[loca][0])-10,int(base[loca][1])-10)
#         baseTupA = (int(base[loca][0])+10,int(base[loca][1])+10)

    color  = [0,0,255]
    
    plt.figure(figsize=(10, 10), edgecolor='r')
    img = np.tile(np.expand_dims(im_t.copy(), axis = 2), (1,1,3))
    tipTup = (int(tip[i][0])-10,int(tip[i][1])-10)
    tipTupA = (int(tip[i][0])+10,int(tip[i][1])+10)
    wristTup = (int(wristTracker[int(t)][0])-10,int(wristTracker[int(t)][1])-10)
    wristTupA = (int(wristTracker[int(t)][0])+10,int(wristTracker[int(t)][1])+10)
    baseTup = (int(base[i][0])-10,int(base[i][1])-10)
    baseTupA = (int(base[i][0])+10,int(base[i][1])+10)

    cv2.rectangle(img, tipTup , tipTupA, color, 4)
    cv2.rectangle(img, wristTup , wristTupA, [0,255,0], 4)
    cv2.rectangle(img, baseTup , baseTupA, color, 4)
    
    plt.imshow(img, 'gray')
    plt.title("Detected FOM".format(int(t)))
    plt.show()

### get only the part during the swing

In [ ]:
swingBox = []
for i in range(len(oldBatBox)):
    if oldBatBox[i][1] in range(80,150)
        swingBox.append(oldBatBox[i])       

batBox = np.array(swingBox)
missingInd = np.arange(80,150).tolist()

for ind in batBox[:,1]:
    if ind in missingInd:
        missingInd.remove(ind)
        
    
# tip = []
# base = []
# for i in range(len(batTip)):
#      if batTip[i][2] in range(80,150):
#         tip.append(batTip[i])
#         base.append(batBase[i])

### Meine alte version von only box bilden

In [ ]:
# Threshold: two times the maximum length of the bat

frame_count = 0
currentBat = batBox[0,0]

onlyBat = batBox.tolist()

cap = cv2.VideoCapture(INPUT_VIDEO_PATH)

# candidates_per_frame = candidates_per_frame[2:]
# for i, cand_list in enumerate(candidates_per_frame):
    
while frame_count<len(candidates_per_frame):
    ret, image_np = cap.read()
    if image_np is None:
        print(frame_count)
        break
    # print(frame_count, len(candidates_per_frame))
    cand_list = candidates_per_frame[frame_count]
    candidates = [[cand.bbox[:2], cand.bbox[2:]] for cand in cand_list]
    if len(candidates)>1:
        candidates = combineOverlapping(candidates)
        # print(candidates)
    
    if len(candidates)>0:
        # print("candidates", candidates, "currentBat", currentBat)
        newBox = closestBox(candidates, currentBat, MAX_DIST_THRESH)
        # print(newBox)
        if newBox.any():
            onlyBat.append([np.array(newBox), int(frame_count)])
            currentBat = newBox
        
        if PLOTTING:
            img = image_np.copy()
            for fom in range(len(candidates)):
                cv2.rectangle(img, tuple(candidates[fom][0]), tuple(candidates[fom][1]),[255,0,0], 4)
            plt.figure(figsize = (20,10))
            plt.imshow(img, 'gray')
            plt.title("Candidates in frame "+str(frame_count))
            plt.show()
    frame_count+=1
    
# print("-----------------")
print("output: [first point of rectange, second point of rectangle, frame]" )
print(onlyBat)